In [5]:
import requests
import urllib
import itertools
from requests.exceptions import ConnectionError, HTTPError, Timeout, TooManyRedirects
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import traceback
import logging
import logging.handlers
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pyodbc
import sqlalchemy as sa
from sqlalchemy import create_engine, event
import json
import time as ti
import datetime
from datetime import datetime
import re

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
global importing, conn
conn = False
con = None
try:
    
    logging.basicConfig(filename = 'CommonConfigLog.log',
                        filemode='a',
                        format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                        datefmt='%H:%M:%S',
                        level=logging.DEBUG)
    logging.info(f'\nLOG START: {datetime.now()}\n')
    
    
    #Opens connection to the SQL Server via the stored config file in the config folder
    def open_connection():
        global server, database, driver, connection, con, dformat
        with open(r'\\GX-ZWESQLPWV342.glblcloud.ad.pwcinternal.com\shared_folders\PeopleDataRecon_DEV\Config\config.json', 'r') as fh:
            config = json.load(fh)
        server = config['server']
        database = config['database']
        driver = config['driver']
        dformat = config['dformat']
        connection = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes'
        con = pyodbc.connect(connection)
        return con
    
    #Tests the connction to the sql server by executing a query that returns the information of the server
    def connection_test():
        cur = con.cursor()
        cur.execute("SELECT @@version")
        row = cur.fetchone()
        print("Connection established to: ",row[0])
        cur.close()
        con.commit()
        logging.info(f"\nConnection established to: {row[0]}\n")
        return 
    
    #simply closes the connection to the server    
    def close_connection():
        con.close()
        return
    
    #checks for the existance of tables passed from get_data() in the SQL server
    def checktables(con, tbl):
        cur = con.cursor()
        query = "SELECT * FROM information_schema.Tables WHERE table_schema = 'import' and table_name = '%s'" % tbl
        cur.execute(query)
        output = cur.fetchall()
        if output == []:
            output = f'\n[import].{tbl} does not exist in the database and will need to be created\n'
            print(output, end = "\r")
            logging.info(output, end = "\r")
            cur.close()
            return False
        else:
            print(output, end = "\r")
            cur.close()
        return True

    #checks the number of existing records of tables passed from get_data() in the SQL server
    def tablecontent(con, tbl):
        cur = con.cursor()
        query = "SELECT COUNT(*) FROM [import].[%s]" % tbl
        cur.execute(query)
        output = cur.fetchone()
        #print(output)
        if output[0] == 0:
            print(f'\n{output[0]} Existing records', end = "\r")
            logging.info(f'\n{output[0]} Existing records\n')
            cur.close()
            return False
        else:
            print(f'\n{output[0]} Existing records', end = "\r")
            logging.info(f'\n{output[0]} Existing records\n')
            cur.close()
            return True

    
    #Removes all previous records from table to prepare for refreshed import data
    def truncate_table(con, tbl):
        try:
            cur = con.cursor()
            query = "TRUNCATE TABLE [import].[%s]" % tbl
            cur.execute(query)
            print(f'\n[import].{tbl} has been succesfully truncated to import new data.')
            logging.info(f'\n[import].{tbl} has been succesfully truncated to import new data.\n')
            cur.close()
            con.commit()
        except Exception as err:
            logging.exception("message")
            cur.close()
            con.rollback()
            #raise err


    #imports dataframe (representation of a table within the SQL Server) into the Commonconfig DB
    def importdata(con, tbl):
        cur = con.cursor()
        engine = sa.create_engine(f'mssql+pyodbc://{server}/{database}?driver={dformat}', fast_executemany = True)
        #pd.io.sql._is_sqlalchemy_connectable(engine)
        df.to_sql(f'{tbl}', engine, index = False, if_exists = 'append', schema = 'import')
        cur.close()
        con.commit()
        return

    
    def getData(CV):
        # convert to config file/table
        global df, recnum
        df = pd.read_excel(r'\\GX-ZWESQLPWV342.glblcloud.ad.pwcinternal.com\shared_folders\Concur\source_file\Concur users.xlsx', skiprows = range(0,2), converters={'Party ID':str}, engine = 'openpyxl')
        recnum = len(df.index)
            
        df = df.loc[~(df['PID'].isnull())]
        
        df['Worker'] = df['First Name'] + ' ' + df['Last Name']
        
        df.rename(columns={'Company Name':'LE_Name',
                           'Employee ID':'EmployeeID',
                           'Email Address':'Email',
                           'Cost Center Name':'CC_Name',
                           'Cost Center Code':'CC_Code',
                           'Employee Grade':'GlobalGrade',
                           'PID':'PartyID'},inplace = True)
        
        df = df[['PartyID',
                 'EmployeeID',
                 'Country',
                 'LE_Name',
                 'Company Code',
                 'Worker',
                 'Email',
                 'Vendor ID',
                 'CC_Name',
                 'CC_Code',
                 'GlobalGrade',
                 'LoS',
                 'Active']]
        
        df = df.reset_index(drop=True)
        
        #display(df)
        recnum = len(df.index)
        
            
        if tablecontent(con, f'{CV}') == False:
            importdata(con, f'{CV}')
            print(f'\n{CV} has been updated with {recnum} records')
            logging.info(f'\n{CV} has been updated with {recnum} records\n')
        else:
            truncate_table(con, f'{CV}')
            importdata(con, f'{CV}')
            print(f'\n{CV} has been updated with {recnum} records')
            logging.info(f'\n{CV} has been updated with {recnum} records\n')
                
            #colnames = list(df)
            
            #display(colnames) 
             #[['Effective_Date','Expiration_Date','Created_Datetime','Last_Modified_Datetime']])
            
            #--fetch column names for table creation and datatypes
            #for colname, dt in itertools.product([df.columns],[df.dtypes]):
                #print(dt)
            #print('\n')
        
        return df
    
    
    #Upon Initial execution, this is the first process that takes place to test connectivity between you and the server
    st = datetime.now()
    open_connection()
    connection_test()
    close_connection()
    
    #Import Process

    global conn
    conn = True
    open_connection()
    #getData(urls[1])
    #close_connection()
    tbl = 'Users'
    if checktables(con, tbl) == False:
        print("Moving to next table.\n", end = "\r")
    else:
        try:
            gstart_time = datetime.now()
            getData(tbl)
            close_connection()
            conn = False
            gend_time = datetime.now()
            print('\nDuration: {}'.format(gend_time - gstart_time))
            logging.info('\nDuration: {}'.format(gend_time - gstart_time))
            print("")
        except (Exception, pyodbc.DatabaseError) as e:
            print("")
            print(e)
            logging.exception('\n')
            logging.exception("message")
            
    et = datetime.now()
    print('Total Execution Duration: {}'.format(et - st),'\n-Import Completed-')
    tt = et - st
    logging.info(f'\nTotal Execution Duration: {tt}\n')
    logging.info('\n-Import Completed-\n')
    
#exceptions 
except (Exception, pyodbc.DatabaseError) as error:
        print(error)
        logging.exception("message")
        pass
    
except requests.exceptions.HTTPError as errh:
    print("Http Error:",  errh)
    conn = False
    logging.exception("message")
    
except requests.exceptions.ConnectionError as errc:
    print("Error Connecting:", errc)
    conn = False
    logging.exception("message")
    
except requests.exceptions.Timeout as errt:
    print("Timeout Error:", errt)
    conn = False
    logging.exception("message")
    
except requests.exceptions.RequestException as erru:
    print("Unidentified Request Exception:", erru)
    conn = False
    logging.exception("message")
        
finally:
    logging.info(f'\nLOG END: {datetime.now()}\n')
    if conn == True:
        close_connection()

Connection established to:  Microsoft SQL Server 2016 (SP2) (KB4052908) - 13.0.5026.0 (X64) 
	Mar 18 2018 09:11:49 
	Copyright (c) Microsoft Corporation
	Enterprise Edition (64-bit) on Windows Server 2016 Datacenter 10.0 <X64> (Build 14393: ) (Hypervisor)

[('Concur', 'import', 'Users', 'BASE TABLE')]
0 Existing records
Users has been updated with 46780 records

Duration: 0:02:39.604392

Total Execution Duration: 0:02:43.901247 
-Import Completed-


In [13]:
with open(r'\\GX-ZWESQLPWV342.glblcloud.ad.pwcinternal.com\shared_folders\PeopleDataRecon_DEV\Config\config.json', 'r') as fh:
            config = json.load(fh)
            print(config)

{'server': 'GX-ZWESQLPWV342.glblcloud.ad.pwcinternal.com', 'database': 'PeopleDataRecon_DEV', 'driver': '{ODBC Driver 17 for SQL Server}', 'dformat': 'ODBC+Driver+17+for+SQL+Server'}


In [16]:
df = pd.read_excel(r'\\GX-ZWESQLPWV342.glblcloud.ad.pwcinternal.com\shared_folders\Concur\source_file\Concur users.xlsx', skiprows = range(0,2), converters={'Party ID':str}, engine = 'openpyxl')
recnum = len(df.index)
            
df = df.loc[(df['PID'].isnull())].reset_index(drop = True)

In [18]:
df = df.reset_index(drop=True)
display(df)

,Country,Company Name,Company Code,First Name,Last Name,Employee ID,Email Address,Cost Center Name,Cost Center Code,Employee Grade,Office Location,PID,Territory,Company,LoS,Vendor ID,Active
0,AUSTRIA,AT PwC Wirtschaftsprüfung,AT01,Adrian,Associate,ATTest1,Adrian.Associate@pwctest.com,ADV Business Recover,AT01000002,Associate,NaN,NaN,NaN,NaN,NaN,NaN,Y
1,AUSTRIA,AT PwC Wirtschaftsprüfung,AT01,Andreas,Associate,ATTest,Andreas.Associate@pwctest.com,ADV Business Recover,AT01000002,Associate,NaN,NaN,AT,AT01,NaN,NaN,Y
2,AUSTRIA,AT PwC Wirtschaftsprüfung,AT01,Martina,Manager,ReportingConsumer01,Martina.Manager@pwctest.com,ADV Business Recover,AT01000002,Manager,NaN,NaN,NaN,NaN,NaN,NaN,Y
3,AUSTRIA,AT PwC Wirtschaftsprüfung,AT01,Michael,Rivas,100802387,michael.rivas@fr.pwc.com,ADV Business Recover,AT01000002,Senior Associate,NaN,NaN,AT,AT18,ADVS,NaN,N
4,AUSTRIA,AT PwC Österreich,AT18,Sophie,Stefan-Eibisberger,100772306,sophie.stefan@pwc.com,ASS Wien,AT18000030,Senior Manager,Wien,NaN,AT,AT18,ASR,50033037,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,SWITZERLAND,PwC AG,CH10,Paulina,Więckowska,100712939,paulina.wieckowska@pwc.com,CH10 Consulting,CH10001001,Senior Associate,NaN,NaN,CH,CH10,ASR,NaN,Y
166,SWITZERLAND,PwC AG,CH10,Radosław,Barski,101182107,radoslaw.barski@pwc.com,CH10 Consulting,CH10001001,Associate,NaN,NaN,CH,CH10,IFS,NaN,Y
167,SWITZERLAND,PwC AG,CH10,Soukaina,test,testID1010,test.user@pwc.ch,CH10 Consulting,CH10001001,Senior Associate,NaN,NaN,CH,CH10,NaN,NaN,Y
168,SWITZERLAND,PwC Liechtenstein,CH11,Michael,Gasser,101232574,michael.a.gasser@pwc.ch,CT East FL,CH11103041,Senior Manager,Ruggell - Industriering 40,NaN,CH,CH11,Tax,50063389,Y


In [4]:
import pyodbc
#Currently determining which datatypes need to be used
#stackoverflow.com/questions/20229441/implications-of-nvarchar-50-vs-nvarchar-max
#stackoverflow.com/questions/35366/what-are-the-main-performance-differences-between-varchar-and-nvarchar-sql-serve
#stackoverflow.com/questions/612430/when-must-we-use-nvarchar-nchar-instead-of-varchar-char-in-sql-server
def create_tables():
    
    """create tables in the SQL Server Database"""
    commands = (
    """ 
    CREATE TABLE [import].[Users](
    [PartyID] [nvarchar](50) NOT NULL,
    [EmployeeID] [nvarchar](150) NOT NULL,
    [Country] [nvarchar](255) NULL,
    [LE_Name] [nvarchar](255) NULL,
    [Company Code] [nvarchar](255) NULL,
    [Worker] [nvarchar](255) NULL,
    [Email] [nvarchar](255) NULL,
    [Vendor ID] [nvarchar](255) NULL,
    [CC_Name] [nvarchar](255) NULL,
    [CC_Code] [nvarchar](255) NULL,
    [GlobalGrade] [nvarchar](255) NULL,
    [LoS] [nvarchar](255) NULL,
    [Active] [nvarchar](255) NULL,
    CONSTRAINT [PK_Users] PRIMARY KEY CLUSTERED (PartyID, EmployeeID)
    )
    """,)
    conn = None
    try:
        server = 'GX-ZWESQLPWV342.glblcloud.ad.pwcinternal.com'
        database = 'Concur'
        conn = pyodbc.connect('Trusted_Connection=yes',DRIVER='{SQL Server}',SERVER=server,DATABASE=database)
        cur = conn.cursor()
        # create table one by one
        for command in commands:
            cur.execute(command)
        # close communication with the SQL Server
        cur.close()
        # commit the changes
        conn.commit()
    except (Exception, pyodbc.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            
if __name__ == '__main__':
    create_tables()